# Cálculo de armónicos esféricos utilizando computación cuántica.   
## TFE: Computación cuántica aplciada al análisis espacial en electroencefalografía   
## Master universitario en computación cuántica (UNIR)   
### Francisco Vidal Requejo   
### Antonio José Ruz Hervás   


In [6]:
%pip install pennylane > o

Note: you may need to restart the kernel to use updated packages.


In [186]:
import pennylane as qml
from pennylane import numpy as np
import math
import sympy as sp
from scipy.special import lpmv
from scipy.special import sph_harm


In [277]:
nCubits = 4
theta = math.pi/5
phi = math.pi/7

In [370]:
dev = qml.device ("default.qubit", wires = nCubits)

@qml.qnode(dev)
def CircuitoCoseno(theta, nCubits = 3):
  """
  Implementar un circuito cuántico con una puerta de ratación sobre el eje Y para todos los cúbits. El mismo ángulo se aplica a todos los cúbits.
  Parámetros:
    theta: Ángulo en radianes que se quiere aplicar en una rotación sobre el eje Y.
    nCubits: Cantidad de cúbits del circuito

  Retorno:
    Array de probabilidades de cada uno de los estados.
  """
  for i in range(nCubits):
    qml.RY(theta, wires = i)
  return qml.probs(wires=range(nCubits))

def CircuitoSeno(theta, nCubits=3):
  """
  Implementar un circuito cuántico con una puerta de rotación de pi/2 - theta sobre el eje Y para todos los cúbits. El mismo ángulo se aplica a todos los cúbits.
  Parámetros:
    theta: Ángulo en radianes que se quiere aplicar como una rotación pi/2 - theta sobre el eje Y.
    nCubits: Cantidad de cúbits del circuito

  Retorno:
    Array de probabilidades de cada uno de los estados.
  """
  return CircuitoCoseno((math.pi/2) - theta, nCubits)

def ExpPhi(phi, m):
  """
  Obtener el valor en coordenadas cartesianas de un número complejo en forma polar de la manera e^{im} a través de dos circuitos cuánticos de un único cúbit utilizando la igualdad e^{im} = cos(im) + i·sen(im) .

  Parámetros:
    phi: el ángulo base
    m: valor a multiplicar al ángulo.

  Retorno:
    Una tupla con el valor real e imagino del número complejo en coordenadas cartesianas.
  """
  C = CircuitoCoseno(phi * m, 1)
  S = CircuitoSeno(phi * m, 1)
  return (C[0] - C[1], S[0] - S[1])

def ProductosPotencias(theta, nCubits):
  """
  Obtener una matriz con la evaluación de los términos cos^k(theta)·sen^l(theta) con l+k = n.
  
  Parámetros:
    theta: el ángulo a aplicar a las funciones trigonométricas.
    nCubits: cantida de cúbits a utilizar, esto es el n = k+l.

  Retorno:
    Matriz con los terminos cos^k(theta)·sen^l(theta) con l+k = n.
  """
  C = CircuitoCoseno(theta, nCubits)
  S = CircuitoSeno(theta, nCubits)
  cos, sen = sp.symbols("cos sen")

def ExtraerTerminosBinomio(probsArray, nCubits = 3):
  """
  Extrae el valor los terminos de la expansión de la potencia de binomios (cos{theta}+bsen{theta})^n a partir de la matriz de probalidades obtenida de un circuito cuántico que aplica la misma rotación a toodos los cúbits.

  Parámetros:
    probArray: el array de probabilidades.
    nCubits: cantidad de cúbits del circuito del que se ha obtenido e array
  
  Retorno:
    Array de tuplas en las que el primer término es un simbolo de sympy con la expresión del témino binomial y el segundo es el valor.
  """
  cos, sen = sp.symbols("cos sen")
  res = []
  for i in range(nCubits +1):
    potenciaCoseno = cos**((nCubits - i)* 2) #como es un valor de problabilidad se multiplica por 2 cada potencia
    potenciaSeno = sen**(i * 2)
    res.append((potenciaCoseno * potenciaSeno, probsArray[2**(i) -1]))
  print(res)
  return res

def PotenciasTrig(terminosBinomio, nCubits = 3, funcion= sp.symbols("cos")):
  """
  Obener el valor del coseno a la 'n' potencia para el ángulo 'theta'. El resultado es una lista con todas las potencias del coseno de 0 a nCubits
  """
  cos, sen = sp.symbols("cos sen")
  c = cos**2
  s = sen**2
  arr = []
  for i in range(nCubits):  #Expansión de los distintos terminos (c+s)^k(c-s)^l con k+l = nCubits.
    potenciaSuma = (c + s)**(nCubits-i - 1)
    potenciaResta = (c - s)**(i + 1)
    productoPotencias = potenciaSuma * potenciaResta
    arr.append(productoPotencias.expand())
  res = []
  for potencia, i in zip(arr, range(nCubits)): #Sustitución de los valores obtenidos del cicuito cuántico en la expansión calculada
    for termino in terminosBinomio:
      potencia = potencia.subs(termino[0], termino[1])
    res.append((funcion**(i+1), potencia))
  return res

def EvalLegendre(theta, ncubits = 4):
  C = CircuitoCoseno(theta, nCubits)
  S = CircuitoSeno(theta, nCubits)
  PolinomiosLegendre = {'P_0_0': 1}
  PolinomiosLegendre['P_1_0']= C[0] + 2*C[1] -2*C[7] -C[15]
  PolinomiosLegendre['P_1_1']= -S[0] - 2*S[1] + 2*S[7] + S[15]
  PolinomiosLegendre['P_2_0']= (3*(C[0] - 2*C[3] + C[15]) - 1)/2
  PolinomiosLegendre['P_2_1']= 3*(C[0] + 2*C[1] - 2*C[7] - C[15])*(-S[0] -2*S[1] + 2*S[7] + S[15])
  PolinomiosLegendre['P_2_2']= 3*(S[0] - 2*S[3] + S[15])
  PolinomiosLegendre['P_3_0']= C[0] - 8*C[1] + 8*C[7] - C[15]
  PolinomiosLegendre['P_3_1']= -(3/2)*(-S[0] + 18*S[1] -18*S[7] + S[15])
  PolinomiosLegendre['P_3_2']= 15*(4*C[1] - 4*C[7])
  PolinomiosLegendre['P_3_3']= -15*(S[0] -2*S[1] + 2*S[7] - S[15])
  PolinomiosLegendre['P_4_0']= (1/8)*(5*C[0] - 140*C[1] + 270*C[3] -140*C[7] + 5*C[15] + 3)
  PolinomiosLegendre['P_4_1']= -(5/2)*(7*(C[0] +2*C[1] -2*C[7] - C[15])*(4*S[1]-4*S[7]) + 3*(C[0]+2*C[1]-2*C[7]-C[15])*(-S[0]-2*S[1]+2*S[7]+S[15]))
  PolinomiosLegendre['P_4_2'] =  (15/2)*(56*C[1] - 112*C[3] + 56*C[7] - S[0] - 28*S[1] + 58*S[3] - 28*S[7] - S[15])
  PolinomiosLegendre['P_4_3'] =  -105*(C[0] + 2*C[1] - 2*C[7] - C[15])*(S[0] - 2*S[1] + 2*S[7] - S[15])
  PolinomiosLegendre['P_4_4'] =  105*(S[0] - 4*S[1] + 6*S[3] - 4*S[7] + S[15])
  return PolinomiosLegendre

def EvalEsfericos(theta, phi, l = 4):
  legendre = EvalLegendre(theta)
  AE = {}
  for i in range(l+1):
    for m in range(-i, i+1):
      print(str(i) + '|' + str(m))
      coef = ((-1)**m)*math.sqrt((((2*i + 1)*math.factorial(i - abs(m))) / (4*math.pi * math.factorial(i+abs(m))))) #Coeficiente de normalización
      e = ExpPhi(phi, m)
      legendre_normalizado = (coef * legendre['P_' + str(i) + '_' + str(abs(m))])
      AE['Y_' + str(i) + '_' + str(m)] = ( legendre_normalizado * e[0], legendre_normalizado * e[1])
  return AE

In [110]:
#c = cos^2(theta/2)
#s = sen^2(theta/2)
cos, sen = sp.symbols("cos sen")
#c, s = sp.symbols("c s")
c = cos**2
s = sen**2
arr = []
nCubits = 4
for i in range(nCubits):
  potenciaSuma = (c + s)**(nCubits-i - 1)
  potenciaResta = (c - s)**(i + 1)
  productoPotencias = potenciaSuma * potenciaResta
  a = productoPotencias.expand()
  print(a)
  arr.append(a)
for x in arr:
  print(x.subs(cos**6*sen**2, 1))

cos**8 + 2*cos**6*sen**2 - 2*cos**2*sen**6 - sen**8
cos**8 - 2*cos**4*sen**4 + sen**8
cos**8 - 2*cos**6*sen**2 + 2*cos**2*sen**6 - sen**8
cos**8 - 4*cos**6*sen**2 + 6*cos**4*sen**4 - 4*cos**2*sen**6 + sen**8
cos**8 - 2*cos**2*sen**6 - sen**8 + 2
cos**8 - 2*cos**4*sen**4 + sen**8
cos**8 + 2*cos**2*sen**6 - sen**8 - 2
cos**8 + 6*cos**4*sen**4 - 4*cos**2*sen**6 + sen**8 - 4


In [348]:
math.cos(phi*4)

-0.22252093395631434

In [371]:
AE = EvalEsfericos(theta, phi)
AE

0|0
1|-1
1|0
1|1
2|-2
2|-1
2|0
2|1
2|2
3|-3
3|-2
3|-1
3|0
3|1
3|2
3|3
4|-4
4|-3
4|-2
4|-1
4|0
4|1
4|2
4|3
4|4


{'Y_0_0': (tensor(1., requires_grad=True), tensor(0., requires_grad=True)),
 'Y_1_-1': (tensor(-0.52957621, requires_grad=True),
  tensor(0.25503046, requires_grad=True)),
 'Y_1_0': (tensor(0.80901699, requires_grad=True),
  tensor(0., requires_grad=True)),
 'Y_1_1': (tensor(-0.52957621, requires_grad=True),
  tensor(-0.25503046, requires_grad=True)),
 'Y_2_-2': (tensor(0.64623129, requires_grad=True),
  tensor(-0.8103484, requires_grad=True)),
 'Y_2_-1': (tensor(-1.28530847, requires_grad=True),
  tensor(0.61897194, requires_grad=True)),
 'Y_2_0': (tensor(0.48176275, requires_grad=True),
  tensor(0., requires_grad=True)),
 'Y_2_1': (tensor(-1.28530847, requires_grad=True),
  tensor(-0.61897194, requires_grad=True)),
 'Y_2_2': (tensor(0.64623129, requires_grad=True),
  tensor(0.8103484, requires_grad=True)),
 'Y_3_-3': (tensor(-0.67782595, requires_grad=True),
  tensor(2.96974951, requires_grad=True)),
 'Y_3_-2': (tensor(2.61406046, requires_grad=True),
  tensor(-3.27792814, requires_g

In [365]:
#m = [0,0,1,0,1,2,0,1,2,3,0,1,2,3,4]
#v = [0,1,1,2,2,2,3,3,3,3,4,4,4,4,4]
m = [0,-1,0,1,-2, 1,0,1,2,-3,-2,-1,0,1,2,3,-4,-3,-2,-1,0,1,2,3,4]
v = [0, 1,1,1, 2, 2,2,2,2, 3, 3, 3,3,3,3,3, 4, 4, 4, 4,4,4,4,4,4]
x = []
for i in range(15):
    x.append(math.cos(theta))

sph_harm(m, v, theta, phi)

array([ 0.28209479+0.j        ,  0.12127512-0.08811153j,
        0.44021565+0.j        , -0.12127512-0.08811153j,
        0.02247113-0.06915901j, -0.24432421-0.17751193j,
        0.45266092+0.j        , -0.24432421-0.17751193j,
        0.02247113+0.06915901j, -0.01053105-0.03241123j,
        0.05356531-0.16485707j,  0.34698932-0.2521025j ,
        0.35596463+0.j        , -0.34698932-0.2521025j ,
        0.05356531+0.16485707j,  0.01053105-0.03241123j,
       -0.0126881 -0.00921844j, -0.02846444-0.08760453j,
        0.09111854-0.28043402j,  0.40130596-0.29156585j,
        0.18091396+0.j        , -0.40130596-0.29156585j,
        0.09111854+0.28043402j,  0.02846444-0.08760453j,
       -0.0126881 +0.00921844j])

In [ ]:
(m, v, x)

In [216]:
4*35

140

In [275]:
#math.sin(theta)**1
#math.cos(theta/2)**8 + 2*math.cos(theta/2)**6*math.sin(theta/2)**2 - 2*math.cos(theta/2)**2*math.sin(theta/2)**6  - math.sin(theta/2)**8
#A = [[a, -b],[b, 1]]
print(abs(-3))

3


In [257]:
cos = sp.symbols("cos")
sen = sp.symbols("sin")
cos2 = sp.symbols("cos2")
sen2 = sp.symbols("sin2")

C0 = cos**8
C1 = cos**6*sen**2
C3 = cos**4*sen**4
C7 = cos**2*sen**6
C15 = sen**8

S0 = cos2**8
S1 = cos2**6*sen2**2
S3 = cos2**4*sen2**4
S7 = cos2**2*sen2**6
S15 = sen2**8

X = (1/8)*(5*C[0] - 140*C[1] + 270*C[3] -140*C[7] + 5*C[7] + 3)
X = (35*(C[0] - 4*C[1] + 6*C[3] - 4*C[7] + C[15]) - 30*(C[0] -2*C[3]  + C[15]) + 3)/8
X = (C[0]+2*C[1]-2*C[7]-C[15])*(-S[0]-2*S[1]+2*S[7]+S[15])

X


tensor(-0.47552826, requires_grad=True)